In [ ]:
import pandas as pd
df = pd.read_csv("all_kindle_review.csv")
df.head()

In [ ]:
df = df[['reviewText','rating']]
df.shape

In [ ]:
df['rating'].value_counts()

In [ ]:
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

In [ ]:
df['reviewText'].str.lower()

In [ ]:

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
from bs4 import BeautifulSoup

In [ ]:

## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
df['reviewText'] = df['reviewText'].apply(lambda x :lemmatize_words(x))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test , y_train , y_test = train_test_split(df['reviewText'],df['rating'],test_size=0.20)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf= tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [ ]:
X_train_bow

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf,y_train)

In [ ]:
y_pred_bow = nb_model_bow.predict(X_test)
y_pred_tfidf = nb_model_tfidf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(accuracy_score(y_test,y_pred_bow))

print(accuracy_score(y_test,y_pred_tfidf))

print(confusion_matrix(y_test,y_pred_bow))

print(confusion_matrix(y_test,y_pred_tfidf))

print(classification_report(y_test,y_pred_bow))

print(classification_report(y_test,y_pred_tfidf))